## Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [126]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from folium.plugins import MarkerCluster

import folium

from src.data_aquisition.overpass import get_pois_near_path, fetch_pois_from_overpass, create_overpass_query, fetch_node_coordinates
from src.visualization.map import map_viz
import src.prompting.prompt_gen as prompting 

In [50]:
TARGET_LAN = 'ru'

### 1. Take hard-coded route as reference

There are some standard formats for saving geo routes:
- GeoJSON (commonly used with web libraries like leaflet and google maps)
- GPX (supported by GPS device and mobile apps, has elevation)
- KML (used by google, good for integration with google maps)
- csv (simple approach)
- SHP (popular for GIS)

In [4]:
kml_file_path = './test_path/path.kml'
gdf = gpd.read_file(kml_file_path, driver='KML')

route_geom = gdf.geometry.iloc[0]


m = map_viz(route = route_geom, buffer=0.0002)
m

### 2. Find objects along the route and get their info

#### 2.1 Specific info (Automatic)

How to find the info?
1. Create a buffer zone around the route
2. Use a geospatial query to find objects within this buffer
3. Retrieve data about the found objects

We can use 
1. OpenStreetMap via Overpass API (free)
2. Google Places API (advantages: ratings, photos, more places, but not free)
3. Foursquare API

In [142]:
pois_near_path = get_pois_near_path(route_geom, buffer_distance=150, max_distance=150)
pois_near_path = [el for el in pois_near_path if el['lat'] is not None and el['lon'] is not None and el['name'] is not None]

radius 745.7969010374758
56.94940812595889
24.10860918604967


https://developers.google.com/maps/documentation/places/web-service/search-along-route

In [143]:
m = map_viz(route=route_geom, markers=pois_near_path)
m

get wikipedia articles about the POIs. Search in english, if no results, then on local language.

In [ ]:
from src.data_aquisition.wikipedia import fetch_wikipedia_article
import src.processing.text_translation as translate

poi_names = [(poi['name']) for poi in pois_near_path]
poi_names_en = [translate.detect_and_translate_to_english(poi) for poi in poi_names]
poi_names_lang = [translate.detect_language(poi) for poi in poi_names]

for i in range(len(pois_near_path)):
    if poi_names_lang[i] == 'en':
        summary = fetch_wikipedia_article(poi_names_en[i], language='en', return_summary=True)
    else:
        summary = fetch_wikipedia_article(poi_names[i], language=poi_names_lang[i], return_summary=True)
        summary = translate.detect_and_translate_to_english(summary)

    pois_near_path[i]['wiki'] = summary

In [67]:
pois_near_path_filtered = [poi for poi in pois_near_path if poi['wiki']!='']

#### Save files for the app

In [ ]:
updated_landmarks = []
for el in landmarks:
    updated_landmarks.append({**{'text':f'{el["address"]} very interesting...'}, **el})

In [ ]:
# prepare json files for the application
import json

routes = [{
    'name':'test_route',
    'coords':[{'latitude':point[1], 'longitude':point[0]} for point in gdf.geometry.iloc[0].coords],
    'landmarks':updated_landmarks
}]


with open('./GeolocationApp/assets/data/routes.json', "w") as f:
    json.dump(routes, f, indent=4)


#### 2.2 General info

1. Use street names
2. Town areas
3. Town
4. Country

### 2.3 Hardcoded landmarks

In [184]:
from test_path.landmarks import landmarks
from src.data_aquisition.wikipedia import fetch_wikipedia_article
import src.processing.text_translation as translate

import pandas as pd

In [185]:
landmarks = pd.DataFrame.from_records(landmarks)
landmarks.head()

,name,address,lat,lon
0,Svētā Pētera baznīca,"Reformācijas Laukums 1, Centra rajons, Rīga, L...",56.947461,24.108570
1,Rātslaukums,"Rātslaukums, Centra rajons, Rīga, 1050, LV-105...",56.947398,24.106729
2,Jauniela,"Jauniela, Riga, LV-1050, Latvia",56.948032,24.105005
3,Jauniela 20,"Jauniela 20, Centra rajons, Rīga, LV-1050, Latvia",56.948619,24.105411
4,Doma laukums,"Central District, Riga, LV-1050, Latvia",56.949524,24.105180


In [ ]:
# center_lat, center_lon = landmarks[['lat','lon']].loc[14].values
# tags = ["historic", "tourism"]
# radius = 50
# query = create_overpass_query(center_lat, center_lon, tags, radius)
# landmark_data = fetch_pois_from_overpass(query)

In [187]:
# gather information from ru, lv wiki articles

landmarks['name_en'] = landmarks['name'].apply(lambda x: translate.detect_and_translate_to(x, 'en', lims=('lv','en')))
landmarks['name_lv'] = landmarks['name'].apply(lambda x: translate.detect_and_translate_to(x, 'lv', lims=('lv','en')))

articles = {}
articles_ru = {}
articles_en = {}

for ind,row in landmarks.iterrows():
    article = fetch_wikipedia_article(row['name'], language='lv', return_summary=False)
    if article == '':
        article = fetch_wikipedia_article(row['name_en'], language='en', return_summary=False)

    # article_ru = translate.detect_and_translate_to(article, target_lan='ru')
    # article_en = translate.detect_and_translate_to(article, target_lan='en')


    # articles[row['name_ru']] = article_ru
    # articles[row['name_ru']] = article_lv

    articles[row['name']] = article
    # articles_ru[row['name']] = article_ru
    # articles_en[row['name']] = article_en


In [196]:
fetch_wikipedia_article('Zviedru vārti Vecrīgā', 'lv', False)

'Zviedru vārti ir 17. gadsimta valsts nozīmes arhitektūras piemineklis Vecrīgā, vienīgie pilsētas aizsargmūra vārti, kas saglabājušies līdz mūsu dienām (savulaik tādi bijuši astoņi).\nPēc bastionu izbūvēšanas vecajam aizsargmūrim vairs nebija praktiskas nozīmes, un XVII gs. pilsētas rāte tam atļāva no ārpuses piebūvēt namus. 1698. gadā Torņa ielā 11 dzīvojamā namā izlauza noslēdzamu caurbrauktuvi, lai savienotu mūra ārpusei piebūvēto kvartālu un zviedru garnizona kazarmas ar iekšpilsētu — t.s. Zviedru vārtus. Vārtus atvēra, saulei lecot, un slēdza, saulei rietot — atslēgas glabājās Rātsnama telpās. Vārtus pārstāja slēgt 19. gadsimta sākumā un vēlāk izņēma no eņģēm. Tagad viņu eja atrodas pie Trokšņu un Aldaru ielas krustojuma.\nVirs vārtiem abās pusēs atrodas lauvas bareljefs kā Zviedrijas karaļa simbols, paši vārti dekorēti ar dolomītu no Sāremā, darbus esot veicis mūrnieks Henriks Henke. Līdzās vārtiem ēkā iebūvētas aizsargmūra bijušā Jirgena torņa atliekas.\n1926. gadā vārtu ēku sāk

In [ ]:
articles

### 3. Utilize LLM to summarize information

Personalization:
Interests
1. history
2. architecture
3. folklore
4. sport
5. music

Storytelling style
1. humorous
2. formal
3. casual

In [84]:
# example
article_text = pois_near_path_filtered[2]['wiki']
user_preferences = {
    "interests": ["history", "architecture"],
    "style": "casual and humorous"
}

In [88]:
narrative = prompting.generate_prompt(article_text, user_preferences)
print(narrative)


    Transform the following text into a tourist guide narrative. Make sure not to exceed 5 sentences.
    User preferences:
    - Interests: history, architecture.
    - Style: casual and humorous.
    Text:
    ""Laima" clock is one of Riga's most famous symbols.It was built according to the wish of the city management for people to prevent work, in 1924, after the project of architect Arthur Medlinger, and was originally popularly known as the Saeima deputy Andrei Veckalns, who put the opposite postulate: the clock needs the city dwellers not to work too fast.In the early 1930s, he brought the advertising of sweets manufacturer "Riegl", but after being added, "Laima" was decorated in 1936 with its symbolism.After World War II, the clock was used for some time as a stand of the authorities' political information.The square at Laima's clock is one of the most popular meetings among Riga residents and guests of Riga.In 1999 it was fully reconstructed by JSC "Laima" funds.In March 2007,

In [ ]:
# manual
gpt_response = 'The Laima Clock is a timeless icon of Riga, ticking since 1924 with more character than your average wristwatch. Originally built to keep workers punctual (or not, depending on who you asked), it later doubled as a sweets ad for Laima, Latvia’s beloved chocolatier, and the name stuck. After surviving wars, propaganda, and decades of wear, it got a chic chocolate-colored makeover in 2017, complete with LED lights and music by Latvian legend Raimonds Pauls. The clock even counted down to Latvia’s 100th birthday—talk about a multitasker! Today, it’s the city’s favorite meet-up spot, perfect for history buffs, architecture fans, and anyone who loves a good photo op.'

### 4. Generate text-to-speech

1. https://elevenlabs.io/text-to-speech (1$ for 30min + commercial use)
2. aws polly (12months free, 1mil char / month, 5 mil char / month)